In [ ]:
import sys
sys.path.append("..") # for sibling import

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import walnut

In [ ]:
device = "cuda" if walnut.cuda.is_available() else "cpu"
device

# Example 4.1

### Convolutional Neural Network: MNIST

The goal of this model is to classify images of hand-written digits.

### Step 1: Prepare data
You will need to download the dataset from https://www.kaggle.com/competitions/digit-recognizer/data and place it into the *data* directory. Only using the official training data for training, validation and testing, since it is just to showcase the framework.

In [ ]:
data = pd.read_csv('../data/mnist/train.csv')
data.head()

In [ ]:
tensor = walnut.df_to_tensor(data)
train, val, test = walnut.preprocessing.split_train_val_test(tensor, ratio_val=0.1, ratio_test=0.1)

In [ ]:
X_train, y_train = train[:, 1:], train[:, 0].astype("int32")
X_val, y_val = val[:, 1:], val[:, 0].astype("int32")
X_test, y_test = test[:, 1:], test[:, 0].astype("int32")

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 1 , 28, -1))
X_val = X_val.reshape((X_val.shape[0], 1, 28, -1))
X_test = X_test.reshape((X_test.shape[0], 1, 28, -1))

print (f'{X_train.shape=}')
print (f'{y_train.shape=}')

print (f'{X_val.shape=}')
print (f'{y_val.shape=}')

print (f'{X_test.shape=}')
print (f'{y_test.shape=}')

Scaling

In [ ]:
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

### Step 2: Build the neural network structure

In [ ]:
import walnut.nn as nn
from walnut.nn.layers import *

model = nn.Sequential([
    Convolution2d(1, 32, kernel_size=(3, 3)),
    ReLU(),
    MaxPooling2d(kernel_size=(2, 2)),
    Flatten(),
    Linear(13*13*32, 256),
    ReLU(),
    Linear(256, 10)
])

model.to_device(device)

In [ ]:
model.compile(
    optimizer=nn.optimizers.Adam(),
    loss_fn=nn.losses.Crossentropy(),
    metric_fn=nn.metrics.get_accuracy
)

In [ ]:
from walnut.nn.analysis import model_summary
model_summary(model, (1, 28, 28))

### Step 3: Train the model

In [ ]:
epochs = 5
batch_size = 350

train_losses, train_scores, _, _ = model.train(X_train, y_train, epochs=epochs, batch_size=batch_size, val_data=(X_val, y_val))

In [ ]:
traces = {
    "train_loss" : train_losses,
    "train_accuracy" : train_scores
}

nn.analysis.plot_curve(traces=traces, figsize=(15, 3), title="train history", x_label="steps", y_label="loss/accuracy")

### Step 4: Evaluate the model

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, batch_size)
print(f'loss {loss:.4f}')
print(f'accuracy {accuracy*100:.2f}')

In [ ]:
predictions = model.predict(X_test, batch_size=batch_size)
nn.analysis.plot_confusion_matrix(predictions, y_test, figsize=(5,5))

### Step 5: Explore the inner workings
Pick a random image from the testing dataset.

In [ ]:
i = random.randint(0, X_test.len - 1)
image = X_test[i].moveaxis(0, -1)  # matplotlib needs the color channel to be the last dim
plt.figure(figsize=(3, 3))
plot = plt.imshow(image.data, cmap='gray')
plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)

Use it to predict a number and show the probability distribution of the outcome.

In [ ]:
image_tensor = walnut.expand_dims(X_test[i], 0)
print(f"correct label: {y_test[i].item()}")

image_tensor.to_device(device)
logits = model(image_tensor).cpu()

print(f"predicted label: {logits.argmax(-1).item()}")
nn.analysis.plot_probabilities(logits, figsize=(6, 3))

Every layer of the model can be accessed to explore their output. Here we iterate over all the kernels of the convolutional layer to explore what they learned to focus on in images.

In [ ]:
channels = {f"{i + 1} {l.__class__.__name__}" : l.y.cpu().data[0] for i, l in enumerate(model.sub_modules[0].sub_modules) if l.__class__.__name__ == "Convolution2d"}
nn.analysis.plot_images(channels, figsize=(40, 40), cmap="gray")